# IBM Tech 2025 Lab


## LAB B: RHELAI & InstructLab


This python notebook is a simple chatbot that uses the requests package to `chat` with a model served on a RHELAI VSI Instance using InstructLab.

If you are unfamiliar with Jupyter Notebook interface, you can quickly go through [documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/notebook-editor.html?context=wx&locale=en&audience=wdp)

### To use this Notebook:
Run each cell below one by one and make sure it completes successfully.

When prompted enter your query and hit Enter. Type 'exit' or 'quit' to terminate the chat loop and then manually select and continue running from next cell

Some example questions are provided.

You may run/repeat the query cells by entering and trying different queries

### Goals

1) Chat with a model served on a RHELAI VSI using InstructLab VLLM
2) Try a document summarization usecase using external assets
3) Chat to expose some limitations of base mistral7b model
4) Chat with a fine-tuned model that has been trained on synthetic generated data (SGD

### LAB B - Exercise 1.a: 
Below is an outline of the code. The code is set up in Python Notebook cells.

1. Set up required libraries
2. Set the variables for accessing the model server.
3. Define the function to chat with model
4. Chat with the model

### LAB B - Exercise 1.b:
1. Load your external data as assets
2. Insert code snippet to read data
3. Chat with the Model

### LAB B - Exercise 2 & 3
1. Review the Model Server URL
2. Chat with the Model

#### 1. Install Pre-requisite libraries

In [1]:
!pip install requests

In [2]:
import requests
import json
import getpass

#### 2. Set the variables for accessing the model server

In [3]:
# Define the model server URL (ensure it's correct for your deployment)
MODEL_SERVER_URL = "https://ilab-base.fs-demo.biz/v1/chat/completions"

# Securely get API key or authentication token if required
API_KEY = "popcorn-applies-flourish"

#### 3. Define function to chat with model using request package

In [4]:
def chat_with_model(prompt):
    headers = {"Content-Type": "application/json"}
    if API_KEY:
        headers["Authorization"] = f"Bearer {API_KEY}"
    
    payload = {
    "model": "rhelai_service_model",
    "messages": [
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt}
    ]
    }
    
    try:
        response = requests.post(MODEL_SERVER_URL, headers=headers, json=payload, verify=True)
        response.raise_for_status()
        data = response.json()
        return data.get("choices", [{}])[0].get("message", {}).get("content", "No response received.")
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return f"Error communicating with model: {e}"


#### 4. Provide a prompt to chat with the served model. Wait for response. Type 'exit' or 'quit' to terminate the chat loop and manually move to next cell.

   Some sample prompts are
   1) Hello, What can you do for me?
   2) What are some of the good places to visit in Singapore?
   3) What is the best way to travel around Singapore?
   4) How hot does it get in Singapore during end of March and should i carry an umbrella?

In [ ]:
# Chat loop
user_input = ""
while True:
    user_input = input("You: Please input a prompt and hit enter")
    print(f"Please wait....")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break
    
    response = chat_with_model(user_input)
    print(f"Model: {response}")

You: Please input a prompt and hit enter What is IBM ?


Please wait....
Error: 503 Server Error: Service Unavailable for url: https://ilab-base.fs-demo.biz/v1/chat/completions
Model: Error communicating with model: 503 Server Error: Service Unavailable for url: https://ilab-base.fs-demo.biz/v1/chat/completions


### Exercise 1.b: Summarize documents

Load your external data as assets and use them in your queries. Here we will load the document into associated COS bucket first and then use its contents to form prompts.

You have two options

1. Use the existing code by copy-pasting the below code block into new cell below

```
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='pQT19GjBZoU33LeoUNiT-dAQf8osQUYpHdzs5lWwNQIR',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'rhelaiinstructlab-donotdelete-pr-tjhwbcblh1zk4y'
object_key = 'RES0217.txt'

# load data of type "text/plain" into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']
```

*OR*

2. Follow the instructions at https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/load-and-access-data.html?context=wx&locale=en&audience=wdp to upload new data and insert code to read it.
   1) From the toolbar, click the Upload asset to project icon (![Image](https://www.ibm.com/docs/en/SSQNUZ_5.1.x/wsj/console/images/find_data_icon.png)) and add your file (Text files upto 5KB).
   2) Click the Code snippets icon ![Image](https://www.ibm.com/docs/en/SSQNUZ_5.1.x/wsj/analyze-data/images/code-snippets-icon.png), click _Read data_, and then _"Select data from your project"_, under _Data Asset_ category, find the file uploaded.
   3) From the _Load as_ drop-down list, select the load option that you prefer(StreamingBody object). If you select Credentials, only file access credentials will be generated.
   4) Click in an empty code cell in your notebook and then click _Insert code to cell_ to insert the generated code. Alternatively, click to _copy_ the generated code to the clipboard and then paste the code into your notebook.

#### 5. Insert code snippet to read data

#### 6. Review the document contents
Update the document = &lt;variable&gt;.read() statement below with the variable into which body of the object is available (from last line of previous cell)

In [ ]:
document = streaming_body_1.read()
print(f"Document read is \n {document.decode('utf-8')}")

#### 7. Use the document read and prompt with it.
  
Sample Prompts:

1) This is a conversation between a doctor and patient, provide a diagnosis and plan the next steps. Explain your response.
2) This is a conversation between a doctor and patient, is there an emergency involved or does this situation need immediate medical attention?
3) Based on the above conversation between a doctor and patient, what are the major concerns discussed and course of treatment advised.

In [ ]:
# Chat loop
while True:
    user_input = input("You: Input a promt and hit Enter")
    print(f"Please wait....")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break
    
    response = chat_with_model(f"{document} \n {user_input}")
    print(f"Model: {response}")

### Exercise 2: Ask about IBM Cloud for Financial Services

The base model is not trained on IBM Cloud for financial services and starts to hallucinate. Prompt to ask about FSCloud

Sample Prompts
1) What is IBM FSCloud and what industry clients is it focussed at?
   > Expected: IBM Cloud for Financial Services is a public cloud environment designed to support the needs of the financial services industry for mission-critical workloads and data protection. It offers compliance capabilities and industry-leading security to help reduce risk for financial institutions, their partners, and FinTechs. 
2) The security control requirements of IBM Cloud for Financial services are based on what?
   > Expected: IBM Cloud for Financial Services is based on 565 control requirements, which are defined in the Control Requirements file. The controls are mapped to various standards such as NIST800-53, ISO 27001, ISO 27017, ISO 27018, and CSA Star.
3) What is IBM Cloud for Financial Services Validated?
   > Expected:IBM Cloud for Financial Services Validated refers to a service that has successfully completed the FS Validation program. It is applicable to internal services, ISVs and vendor solutions and offerings.
   
Notice how the responses are vague and in some cases downright incorrect. The base model is hallucinating.

#### 8. Set the endpoint for VSI instance serving base model and chat with the model

In [ ]:
# Define the base model server URL (ensure it's correct for your deployment)
MODEL_SERVER_URL = "https://ilab-base.fs-demo.biz/v1/chat/completions"

In [ ]:
# Chat loop
while True:
    user_input = input("You: Please input a prompt and hit Enter")
    print(f"Please wait....")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break
    
    response = chat_with_model(f"{user_input}")
    print(f"Model: {response}")

### Exercise 3: Fine-tuned model

The base model is fine-tuned on a knowledge base of fscloud. The fine-tuned model is served on a different endpoint. 

Repeat the prompts to learn about IBM Cloud for Financial Services.

In [ ]:
#### 9. Set the endpoint for VSI instance serving the fine tuned model and chat with the model

In [ ]:
# Define the base model server URL (ensure it's correct for your deployment)
MODEL_SERVER_URL = "https://ilab-tuned.fs-demo.biz/v1/chat/completions"

In [ ]:
# Chat loop
while True:
    user_input = input("You: Please input a prompt and hit Enter")
    print(f"Please wait....")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break
    
    response = chat_with_model(f"{user_input}")
    print(f"Model: {response}")